# **저시력자를 위한 원화 화폐 분류**

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install gdown

In [3]:
import zipfile, gdown
import os, json
import glob, shutil
import random
import yaml
import torch

### (2) 데이터셋 불러오기

In [4]:
file_name = "money_dataset.zip"
output = "/content/drive/MyDrive/Colab Notebooks/KTAIVLE/Miniproject3/" + file_name

In [5]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(output)
# 데이터셋 압축 해제
money_data.extractall('/content/Dataset/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---

In [6]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [7]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/Dataset/'

In [8]:
train_ratio = 0.8
folder_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']

for folder in folder_list:
    images_folder = os.path.join(data_path, folder)
    image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]

    # 데이터 분할 인덱스 계산
    train_samples = int(len(image_files) * train_ratio)

    train_images_dir = os.path.join(data_path, 'images', 'train')
    train_labels_dir= os.path.join(data_path, 'labels', 'train')
    validation_images_dir = os.path.join(data_path, 'images', 'val')
    validation_labels_dir= os.path.join(data_path, 'labels', 'val')

    for i, image_file in enumerate(image_files):
        if i < train_samples:
            shutil.copy(os.path.join(images_folder, image_file), os.path.join(train_images_dir, image_file))
            label_file = image_file.replace('.jpg', '.json')
            shutil.copy(os.path.join(images_folder, label_file), os.path.join(train_labels_dir, label_file))
        else:
            shutil.copy(os.path.join(images_folder, image_file), os.path.join(validation_images_dir, image_file))
            label_file = image_file.replace('.jpg', '.json')
            shutil.copy(os.path.join(images_folder, label_file), os.path.join(validation_labels_dir, label_file))


In [9]:
images_train = '/content/Dataset/images/train'
images_val = '/content/Dataset/images/val'

print(len(os.listdir(images_train)),len(os.listdir(images_val)))

4172 1046


### (2) json에서 정보 추출


In [10]:
 class_name_to_number = {
        'Fifty': 1,
        'Fifty_Thousand': 7,
        'Five_Hundred': 3,
        'Five_Thousand': 5,
        'Hundred': 2,
        'Ten': 0,
        'Ten_Thousand': 6,
        'Thousand': 4
    }

In [11]:
json_folder = '/content/Dataset/labels/'
yolo_txt_folder = '/content/Dataset/labels/'
resize_ratio = 1 / 5

# train & val 폴더, JSON 파일
for subset in ['train', 'val']:

    json_path = os.path.join(json_folder, subset)
    os.makedirs(os.path.join(yolo_txt_folder, subset), exist_ok=True)
    json_files = [f for f in os.listdir(json_path) if f.endswith('.json')]

    # JSON 처리
    for json_file in json_files:
        with open(os.path.join(json_path, json_file), 'r') as f:
            data = json.load(f)

            # 클래스 이름
            class_name=data['shapes'][0]['label']
            class_name = class_name.replace('_front', '').replace('_back', '')
            if class_name in class_name_to_number:
                    class_name = class_name_to_number[class_name]

            # 위치 & 박스
            x1, y1 = data['shapes'][0]['points'][0]
            x2, y2 = data['shapes'][0]['points'][1]
            x_center = (x1 + x2) / 2  / data['imageWidth']
            y_center = (y1 + y2) / 2  /data['imageHeight']
            width = (x2 - x1)  / data['imageWidth']
            height = (y2 - y1) / data['imageHeight']

            # 라벨 생성
            yolo_label = f"{class_name} {x_center} {y_center} {width} {height}"

            #  txt 파일로 저장
            txt_file_name = json_file.replace('.json', '.txt')
            with open(os.path.join(yolo_txt_folder, subset, txt_file_name), 'w') as txt_f:
                txt_f.write(yolo_label)



### (3) 데이터셋 정보가 담긴 파일(yaml) 생성



In [12]:
won_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}

In [13]:
data = {
    'train': '/content/Dataset/images/train',
    'val':  '/content/Dataset/images/val',

    # 'train': os.path.join('/content/Dataset/', "image_train.txt"),
    # 'val': os.path.join ('/content/Dataset/', "image_val.txt"),
    'names': won_dict,
    'nc': 8
}

# YAML 파일로 저장
yaml_file_path = '/content/Dataset/config.yaml'
with open(yaml_file_path, 'w',encoding="utf-8") as yaml_file:
    yaml.dump(data, yaml_file, default_flow_style=False)


## 3.모델링

### (1) 모델 라이브러리 설치
---

In [14]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16013, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 16013 (delta 25), reused 23 (delta 13), pack-reused 15967
Receiving objects: 100% (16013/16013), 14.66 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (10987/10987), done.
/content/yolov5


In [15]:
%pip install -qr requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.7/644.7 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


In [16]:
from ultralytics import YOLO

In [17]:
model = YOLO(model='yolov5s.pt', task='detect')

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

100%|██████████| 17.7M/17.7M [00:00<00:00, 137MB/s] 


In [19]:
# 가중치 파일
model = YOLO(model='/content/drive/MyDrive/Colab Notebooks/KTAIVLE/Miniproject3/best1.pt',task='detect')

In [ ]:
# # YOLOv5 🚀 by Ultralytics, AGPL-3.0 license
# # Hyperparameters for low-augmentation COCO training from scratch
# # python train.py --batch 64 --cfg yolov5n6.yaml --weights '' --data coco.yaml --img 640 --epochs 300 --linear
# # See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

# lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
# lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf)
# momentum: 0.937  # SGD momentum/Adam beta1
# weight_decay: 0.0005  # optimizer weight decay 5e-4
# warmup_epochs: 3.0  # warmup epochs (fractions ok)
# warmup_momentum: 0.8  # warmup initial momentum
# warmup_bias_lr: 0.1  # warmup initial bias lr
# box: 0.05  # box loss gain
# cls: 0.5  # cls loss gain
# cls_pw: 1.0  # cls BCELoss positive_weight
# obj: 1.0  # obj loss gain (scale with pixels)
# obj_pw: 1.0  # obj BCELoss positive_weight
# iou_t: 0.20  # IoU training threshold
# anchor_t: 4.0  # anchor-multiple threshold
# # anchors: 3  # anchors per output layer (0 to ignore)
# fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
# hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
# hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
# hsv_v: 0.4  # image HSV-Value augmentation (fraction)
# degrees: 0.0  # image rotation (+/- deg)
# translate: 0.1  # image translation (+/- fraction)
# scale: 0.5  # image scale (+/- gain)
# shear: 0.0  # image shear (+/- deg)
# perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
# flipud: 0.0  # image flip up-down (probability)
# fliplr: 0.5  # image flip left-right (probability)
# mosaic: 1.0  # image mosaic (probability)
# mixup: 0.0  # image mixup (probability)
# copy_paste: 0.0  # segment copy-paste (probability)


In [20]:
model.train(data='/content/Dataset/config.yaml',
            epochs=3,
            patience=7,
            pretrained=True,
            verbose=True,
            seed=2023,
            save=True,
            project='trained',
            degrees=1
            )

Ultralytics YOLOv8.0.201 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/Colab Notebooks/KTAIVLE/Miniproject3/best1.pt, data=/content/Dataset/config.yaml, epochs=3, patience=7, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=trained, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2023, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retin

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7abdfce38ca0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,

In [25]:
from PIL import Image

In [29]:
# 평가
test_data_folder = '/content/drive/MyDrive/Colab Notebooks/KTAIVLE/Miniproject3/test_data/'

png_files = [file for file in os.listdir(test_data_folder) if file.endswith('.png')]
for png_file in png_files:
    source_path = os.path.join(test_data_folder, png_file)
    results = model.predict(source=source_path, save=True)




image 1/1 /content/drive/MyDrive/Colab Notebooks/KTAIVLE/Miniproject3/test_data/1.png: 640x640 1 100, 15.9ms
Speed: 2.7ms preprocess, 15.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to trained/train38

image 1/1 /content/drive/MyDrive/Colab Notebooks/KTAIVLE/Miniproject3/test_data/2.png: 640x512 1 10, 4 50s, 1 100, 12.4ms
Speed: 1.9ms preprocess, 12.4ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 512)
Results saved to trained/train39

image 1/1 /content/drive/MyDrive/Colab Notebooks/KTAIVLE/Miniproject3/test_data/6.png: 640x640 2 50s, 15.8ms
Speed: 2.4ms preprocess, 15.8ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to trained/train40

image 1/1 /content/drive/MyDrive/Colab Notebooks/KTAIVLE/Miniproject3/test_data/8.png: 512x640 1 1000, 1 10000, 13.4ms
Speed: 1.8ms preprocess, 13.4ms inference, 1.8ms postprocess per image at shape (1, 3, 512, 640)
Results saved to trained/train41

image 1/1 /conte

In [ ]:
# results = model.predict(source='/content/drive/MyDrive/Colab Notebooks/KTAIVLE/Miniproject3/test_data/9.jpg',
#                         save=True,
#                         )

In [ ]:
# won_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}